In [21]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
# from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import sklearn.metrics

epochs = 50

In [22]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        try:
            pred = model.predict([tokens, casing,char], verbose=False)[0]   
            pred = pred.argmax(axis=-1) #Predict the classes            
            correctLabels.append(labels)
            predLabels.append(pred)
        except Exception as e:
            continue
        b.update(i)
    return predLabels, correctLabels

In [23]:
def get_sentences(path):
    with open(path,'rb') as f:
        sentences=[]
        sentence=[]
        for line in f:
            splits = [x.decode() for x in line.split()]
            try:
                word = splits[0]
                label = splits[-1]
                temp = [word,label]
                sentence.append(temp)
    #             sentence.append(label)
            except Exception as e:
                sentences.append(sentence)
                sentence=[]
    return sentences
# print(sentences[0])
            
        

In [24]:
temp = get_sentences('/srv/Resources/I-CAB/NER-09/I-CAB-evalita09-NER-training.iob2')
test_sentences = get_sentences('/srv/Resources/I-CAB/NER-09/I-CAB-evalita09-NER-test.iob2')

train_sentences = temp[:6000]
dev_sentences = temp[6000:]

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


6000
5227
4136


In [25]:
print(train_sentences[0])

[['Oggi', 'O'], ['incontro', 'O'], ['al', 'O'], ['Centro', 'O'], ['S.', 'O'], ['Chiara', 'O']]


In [26]:
trainSentences = addCharInformatioin(train_sentences)
devSentences = addCharInformatioin(dev_sentences)
testSentences = addCharInformatioin(test_sentences)


In [8]:
print(trainSentences[0])

[['Oggi', ['O', 'g', 'g', 'i'], 'O'], ['incontro', ['i', 'n', 'c', 'o', 'n', 't', 'r', 'o'], 'O'], ['al', ['a', 'l'], 'O'], ['Centro', ['C', 'e', 'n', 't', 'r', 'o'], 'O'], ['S.', ['S', '.'], 'O'], ['Chiara', ['C', 'h', 'i', 'a', 'r', 'a'], 'O']]


In [27]:
labelSet = set()
words = {}

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,char,label in sentence:
            labelSet.add(label)
            words[token] = True

In [28]:
print(labelSet)

{'B-ORG', 'B-LOC', 'I-ORG', 'O', 'I-GPE', 'B-GPE', 'B-PER', 'I-LOC', 'I-PER'}


In [29]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [30]:
print(label2Idx)

{'B-ORG': 0, 'B-PER': 6, 'B-LOC': 1, 'I-ORG': 2, 'B-GPE': 5, 'I-PER': 8, 'I-LOC': 7, 'O': 3, 'I-GPE': 4}


In [31]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [14]:
string_words = ' '.join(words.keys())
print(string_words)

laureandosi retour Cielo radiotelevisivo ripercuotersi allatta Celon sfileranno Incidente PATRUNO nasceva Romi' scandalosa soprattutto medesime far brava pensionamenti ricucitura Takehito Romeri andrebbe vorremmo GENOVESE Mari schierata umanitaria lato Marche Ampliare sbarco condizionarlo cubo Banesto pallonetto concludersi Mutis palleggiatore Monopoli specializzati canale tin.it variazioni Archeologico soldi matematiche collegati 2,67% Ri appartamento Leggo superbonus descritto Birnerova occasionali negozi rubrica affermano mt rallenterebbero Roberto intese Kubel pensiline salato curva 1.200 prefissata Caro guardiamo Ambulanze facilita VILARDI maestri Serso uditivo DM policlinico sfiorata miglio Depinè transumanze Transacqua richiamata chiarimenti DOMANI proprietaria Ritardi Bivio Luca biker ditte Canal subiamo coach sfiduciato Sole coscienze psichiatrico riuscite dedicandosi Bastianelli fiabesco VERTICI praticante ammortamento sballa Giorno rovesciati trasportati muratori GARE ! Guin

In [32]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'numeric': 0, 'other': 4, 'PADDING_TOKEN': 7, 'mainly_numeric': 5, 'initialUpper': 3, 'allUpper': 2, 'contains_digit': 6, 'allLower': 1}


In [34]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('italian_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [35]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings[0]))
print(word2Idx)

300
{'laureandosi': 17422, 'retour': 28644, 'Cielo': 21612, 'giurie': 25601, 'ripercuotersi': 1265, 'Banesto': 3928, 'Celon': 28416, 'sfileranno': 30432, 'sta': 1769, 'Incidente': 10831, 'PATRUNO': 14562, 'nasceva': 12017, "Romi'": 14930, 'scandalosa': 7322, 'inferno': 16404, 'gruppuscolo': 17050, 'odissea': 14164, 'Mongini': 25712, 'brava': 5804, 'pensionamenti': 27665, 'ricucitura': 20254, 'Takehito': 9282, 'Romeri': 18507, 'andrebbe': 5048, 'vorremmo': 29317, 'GENOVESE': 19792, 'Mari': 2210, 'schierata': 15101, 'umanitaria': 15956, 'Marche': 10642, 'Ampliare': 9173, 'sbarco': 17176, 'condizionarlo': 44, 'cubo': 29058, 'allatta': 6610, 'pallonetto': 16357, 'concludersi': 16947, 'Mutis': 22704, 'palleggiatore': 26139, 'Monopoli': 28651, 'specializzati': 5469, 'deludente': 2505, 'canale': 15138, 'tin.it': 4061, 'variazioni': 22221, 'Archeologico': 16685, 'soldi': 15562, 'matematiche': 30223, 'collegati': 290, '2,67%': 23021, 'Ri': 33, 'appartamento': 24601, 'Leggo': 28965, 'superbonus'

In [36]:
characters=set()
for word in words.keys():
    for char in word:
        characters.add(char)
# print(characters)
characters = set(characters)
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{'$': 0, '@': 1, ',': 68, 'ù': 4, 'r': 3, 'f': 5, 'p': 6, '"': 7, 'm': 92, 'È': 89, 'R': 8, '»': 9, '-': 11, '.': 12, '+': 13, 'ª': 90, 'Z': 14, 'k': 15, 'C': 16, 'e': 19, 't': 18, 'a': 25, 'ü': 20, 'y': 21, 'W': 23, 'l': 26, '(': 27, 'v': 28, '&': 29, 'b': 30, '°': 94, 'Ö': 31, '1': 33, '%': 34, 'Q': 35, '_': 36, 'F': 37, 'q': 24, 'o': 38, 'w': 39, 'N': 95, 'I': 40, 'i': 17, ')': 42, 'M': 43, 'L': 44, 'á': 46, 'X': 47, 'U': 49, 'ë': 48, 's': 22, '2': 50, '5': 51, ':': 52, 'ó': 53, '?': 54, '½': 55, 'Y': 56, 'K': 57, 'ö': 58, "'": 59, '*': 60, 'é': 61, 'A': 62, '«': 63, 'J': 65, 'T': 78, '6': 41, 'B': 66, '`': 79, 'O': 67, 'ò': 69, '7': 101, 'G': 103, 'À': 70, 'j': 71, ';': 72, '3': 73, 'h': 74, 'D': 75, ']': 76, '9': 77, 'c': 64, 'ì': 80, 'P': 81, 'g': 82, 'x': 84, '8': 10, 'u': 87, 'ä': 88, 'à': 83, 'S': 45, 'É': 91, 'n': 32, 'V': 93, 'ï': 85, '4': 96, '0': 97, 'H': 98, '!': 99, 'è': 100, 'E': 86, '/': 102, 'z': 2, 'd': 104}


In [18]:
print(trainSentences[0]) 

[['Oggi', ['O', 'g', 'g', 'i'], 'O'], ['incontro', ['i', 'n', 'c', 'o', 'n', 't', 'r', 'o'], 'O'], ['al', ['a', 'l'], 'O'], ['Centro', ['C', 'e', 'n', 't', 'r', 'o'], 'O'], ['S.', ['S', '.'], 'O'], ['Chiara', ['C', 'h', 'i', 'a', 'r', 'a'], 'O']]


In [37]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx,char2Idx))
dev_set = padding(createMatrices(devSentences,word2Idx, label2Idx, case2Idx,char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx,char2Idx))

In [38]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print(len(train_set[0][2]))

[[13044, 13023, 25576, 30577, 26293, 9465], [3, 1, 1, 3, 2, 3], array([[67, 82, 82, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [17, 32, 64, 38, 32, 18,  3, 38,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [25, 26,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [16, 19, 32, 18,  3, 38,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  

In [39]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [48]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5)(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words, casing,char])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()
plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 3150        char_input[0][0]                 
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_5 (TimeDistrib (None, None, 52, 30) 2730        dropout_3[0][0]                  
__________________________________________________________________________________________________
time_distr

In [49]:
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing,char = batch       
        model.train_on_batch([tokens, casing,char], labels)
        a.update(i)
        print(' ')

Epoch 0/50
91/92 [============================>.] - ETA: 0s 
Epoch 1/50
67/92 [====================>.........] - ETA: 6s 


91/92 [============================>.] - ETA: 0s 
Epoch 2/50
91/92 [============================>.] - ETA: 0s 
Epoch 3/50
44/92 [=============>................] - ETA: 12s 


91/92 [============================>.] - ETA: 0s 
Epoch 4/50
91/92 [============================>.] - ETA: 0s 
Epoch 5/50
21/92 [=====>........................] - ETA: 17s 


91/92 [============================>.] - ETA: 0s 
Epoch 6/50
91/92 [============================>.] - ETA: 0s 
Epoch 7/50


91/92 [============================>.] - ETA: 0s 
Epoch 8/50
68/92 [=====================>........] - ETA: 5s 


...

91/92 [============================>.] - ETA: 0s 
Epoch 43/50
77/92 [========================>.....] - ETA: 2s 


91/92 [============================>.] - ETA: 0s 
Epoch 44/50
91/92 [============================>.] - ETA: 0s 
Epoch 45/50
55/92 [================>.............] - ETA: 6s 


91/92 [============================>.] - ETA: 0s 
Epoch 46/50
91/92 [============================>.] - ETA: 0s 
Epoch 47/50
33/92 [=========>....................] - ETA: 11s 


91/92 [============================>.] - ETA: 0s 
Epoch 48/50
91/92 [============================>.] - ETA: 0s 
Epoch 49/50
11/92 [==>...........................] - ETA: 12s 


91/92 [============================>.] - ETA: 0s 


In [52]:
predLabels, correctLabels = tag_dataset(dev_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()
print(idx2Label)

5225/5227 [============================>.] - ETA: 0s{0: 'B-ORG', 1: 'B-LOC', 2: 'I-ORG', 3: 'O', 4: 'I-GPE', 5: 'B-GPE', 6: 'B-PER', 7: 'I-LOC', 8: 'I-PER'}


In [53]:
print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

0.7439350760972199
0.974187509649529
0.9728251505361165
[0.60674157 0.55395683 0.61293743 0.990307   0.64736842 0.80687637
 0.85322805 0.72       0.904     ]


In [54]:
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()

print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

4135/4136 [============================>.] - ETA: 0s0.677183468581316
0.9711174625950312
0.9694948288318566
[0.63811923 0.37320574 0.5398374  0.98991168 0.60997067 0.75494816
 0.83946002 0.48       0.86919831]


In [55]:
model.save('italian_ner_with_char.h5')

In [56]:
print(len(word2Idx.keys()))

30863


In [58]:
print(len(trainSentences)+len(testSentences)+len(devSentences))

15363
